In [4]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
try:
  from prophet import Prophet
except:
  !pip install prophet
  from prophet import Prophet
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Data_Glacier/Week7/forecasting_case_study.csv')

df['date'] = pd.to_datetime(df['date'])

df['Price Discount (%)'] = df['Price Discount (%)'].astype(str)
df['Price Discount (%)'] = df['Price Discount (%)'].str.replace('%', '').astype(float)
df['Price Discount (%)'] = df['Price Discount (%)'].round(2)
df['Price Discount (%)'] = df['Price Discount (%)']*100

df['Price Discount (%)'] = df['Price Discount (%)'].replace('', np.nan).astype(float).fillna(0).astype(int)

df['Google_Mobility']= df['Google_Mobility'].round(2)

# Extract features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.week
df['day_of_week'] = df['date'].dt.dayofweek
for lag in [1, 2, 3]:
    df[f'sales_lag_{lag}'] = df.groupby('Product')['Sales'].shift(lag)
df['rolling_mean_3'] = df.groupby('Product')['Sales'].transform(lambda x: x.rolling(window=3).mean())
df['rolling_std_3'] = df.groupby('Product')['Sales'].transform(lambda x: x.rolling(window=3).std())

# Normalize
scaler = StandardScaler()
numerical_cols = ['Price Discount (%)', 'Google_Mobility', 'sales_lag_1', 'sales_lag_2', 'sales_lag_3', 'rolling_mean_3', 'rolling_std_3']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df.dropna(inplace=True)

# Synthetic data generation using ARIMA and Decomposition
synthetic_data = []
skus = df['Product'].unique()
np.random.seed(42)

for sku in skus:
    sku_data = df[df['Product'] == sku]
    N = len(sku_data)  # The length of the original series for each SKU

    series = sku_data['Sales'].values

    # ARIMA based synthetic generation
    arima = ARIMA(series, order=(5,1,0))
    arima_fitted = arima.fit()

    arima_forecast = arima_fitted.forecast(steps=7)
    synthetic_series_arima = np.concatenate([series, arima_forecast])

    # Choose decomposition model based on data
    if np.any(series <= 0):
        decompose_model = 'additive'
    else:
        decompose_model = 'multiplicative'

    # Decomposition based synthetic generation
    result = seasonal_decompose(series, model=decompose_model, extrapolate_trend='freq', period=12)
    residual = result.resid
    seasonal = result.seasonal
    trend = result.trend
    # Extract mean value to extend the decomposed series
    mean_value = np.mean(series)
    synthetic_series_decompose = np.concatenate([
        trend + np.random.choice(residual, size=N, replace=True) + seasonal,
        np.full(7, mean_value)  # extend by 7 days using mean value
        ])[:len(series) + 7]

    # Combine the synthetic series
    combined_synthetic_series = (synthetic_series_arima + synthetic_series_decompose) / 2

    # Append to synthetic data
    synthetic_data.append(pd.DataFrame({
        #'Product': [sku] * N,
        'Product': [sku] * (len(series) + 7),
        'Sales': combined_synthetic_series,
        #'date': np.tile(sku_data['date'].values, (int(N/len(series)), )),
        'date': list(sku_data['date'].values) + [sku_data['date'].iloc[-1] + pd.Timedelta(days=x) for x in range(1, 8)],
        #'date': combined_dates,
        #'year': np.tile(sku_data['year'].values, (int(N/len(series)), )),
        #'month': np.tile(sku_data['month'].values, (int(N/len(series)), )),
        #'week': np.tile(sku_data['week'].values, (int(N/len(series)), )),
        #'day_of_week': np.tile(sku_data['day_of_week'].values, (int(N/len(series)), )),
        'year': list(sku_data['year'].values) + [sku_data['date'].iloc[-1].year] * 7,
        'month': list(sku_data['month'].values) + [sku_data['date'].iloc[-1].month] * 7,
        'week': list(sku_data['week'].values) + [sku_data['date'].iloc[-1].week] * 7,
        'day_of_week': list(sku_data['day_of_week'].values) + [(sku_data['day_of_week'].iloc[-1] + x) % 7 for x in range(1, 8)]

    }))

synthetic_df = pd.concat(synthetic_data)
new_df = pd.concat([df, synthetic_df])

import numpy as np

def mean_absolute_percentage_error(y_test, ensemble_forecast):
    """
    Calculate Mean Absolute Percentage Error (MAPE).

    Parameters:
    y_test (array-like): True target values.
    ensemble_forecast (array-like): Predicted target values.

    Returns:
    float: MAPE score.
    """
    y_true, y_pred = np.array(y_test), np.array(ensemble_forecast)
    return np.mean(np.abs((y_test - ensemble_forecast) / y_test)) * 100

from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
import numpy as np
import warnings
try:
  from bayes_opt import BayesianOptimization
except:
  !pip install bayesian-optimization
  from bayes_opt import BayesianOptimization

from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')  # Suppress warnings

def custom_mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    diff = np.abs((y_true - y_pred) / np.where(y_true == 0, 1, y_true))
    return 100 * np.mean(diff)

def create_features(data):
    data['lag_1'] = data['Sales'].shift(1)
    data['lag_2'] = data['Sales'].shift(2)
    data['lag_3'] = data['Sales'].shift(3)
    data['rolling_mean'] = data['Sales'].rolling(window=3).mean()
    return data

def arima_evaluate(p, d, q):
    try:
        model = ARIMA(y_train, order=(int(p), int(d), int(q)))
        model_fitted = model.fit()
        forecast = model_fitted.forecast(steps=len(test))
        return -mean_squared_error(y_test, forecast)
    except:
        return float('-inf')

def rfr_evaluate(n_estimators, max_depth):
    model = RandomForestRegressor(n_estimators=int(n_estimators), max_depth=int(max_depth))
    model.fit(X_train_scaled, y_train)
    predictions = model.predict(X_test_scaled)
    return -mean_squared_error(y_test, predictions)

def gbm_evaluate(n_estimators, learning_rate, max_depth):
    model = GradientBoostingRegressor(n_estimators=int(n_estimators), learning_rate=learning_rate, max_depth=int(max_depth))
    model.fit(X_train_scaled, y_train)
    predictions = model.predict(X_test_scaled)
    return -mean_squared_error(y_test, predictions)

# Implement the Prophet evaluator function for Bayesian Optimization
def prophet_evaluate(seasonality_prior_scale, holidays_prior_scale, changepoint_prior_scale):
    try:
        prophet_data = pd.DataFrame({'ds': train['date'], 'y': y_train})
        model_prophet = Prophet(
            yearly_seasonality=True,
            daily_seasonality=False,
            weekly_seasonality=True,
            seasonality_prior_scale=seasonality_prior_scale,
            holidays_prior_scale=holidays_prior_scale,
            changepoint_prior_scale=changepoint_prior_scale
        )
        model_prophet.fit(prophet_data)
        future = model_prophet.make_future_dataframe(periods=len(test))
        forecast_prophet = model_prophet.predict(future)
        prophet_forecast = forecast_prophet['yhat'][-len(test):].values
        return -mean_squared_error(y_test, prophet_forecast)
    except:
        return float('-inf')

arima_bounds = {
    'p': (0, 5),
    'd': (0, 3),
    'q': (0, 5)
}

rfr_bounds = {
    'n_estimators': (50, 500),
    'max_depth': (1, 10)
}

gbm_bounds = {
    'n_estimators': (50, 500),
    'learning_rate': (0.01, 0.8),
    'max_depth': (1, 10)
}

prophet_bounds = {
    'seasonality_prior_scale': (0.01, 10),
    'holidays_prior_scale': (0.01, 10),
    'changepoint_prior_scale': (0.001, 0.5)
}

results = {}
ensemble_results = {}
skus = df['Product'].unique()
scaler = MinMaxScaler()

for sku in skus:
    print(f"Processing SKU: {sku}")

    sku_data = df[df['Product'] == sku]
    sku_data = create_features(sku_data)

    train = sku_data[sku_data['date'] <= '2020-09-30'].dropna()
    test = sku_data[sku_data['date'] > '2020-09-30']

    X_train = train.drop(['Sales', 'Product', 'date'], axis=1)
    y_train = train['Sales']
    X_test = test.drop(['Sales', 'Product', 'date'], axis=1)
    y_test = test['Sales']

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Handle NaNs
    X_train_scaled = np.nan_to_num(X_train_scaled)
    X_test_scaled = np.nan_to_num(X_test_scaled)

    # ARIMA Optimization
    optimizer = BayesianOptimization(arima_evaluate, arima_bounds, random_state=27)
    optimizer.maximize(init_points=5, n_iter=100)
    best_params = optimizer.max['params']
    arima = ARIMA(y_train, order=(int(best_params['p']), int(best_params['d']), int(best_params['q'])))
    arima_fitted = arima.fit()
    arima_forecast = arima_fitted.forecast(steps=len(test))

    # RandomForestRegressor Optimization
    optimizer = BayesianOptimization(rfr_evaluate, rfr_bounds, random_state=27)
    optimizer.maximize(init_points=5, n_iter=100)
    best_params = optimizer.max['params']
    rfr = RandomForestRegressor(n_estimators=int(best_params['n_estimators']), max_depth=int(best_params['max_depth']))
    rfr.fit(X_train_scaled, y_train)
    rfr_forecast = rfr.predict(X_test_scaled)

    # GradientBoostingRegressor Optimization
    optimizer = BayesianOptimization(gbm_evaluate, gbm_bounds, random_state=27)
    optimizer.maximize(init_points=5, n_iter=100)
    best_params = optimizer.max['params']
    gbm = GradientBoostingRegressor(n_estimators=int(best_params['n_estimators']), learning_rate=best_params['learning_rate'], max_depth=int(best_params['max_depth']))
    gbm.fit(X_train_scaled, y_train)
    gbm_forecast = gbm.predict(X_test_scaled)

    # Prophet Optimization
    optimizer = BayesianOptimization(prophet_evaluate, prophet_bounds, random_state=27)
    optimizer.maximize(init_points=5, n_iter=100)
    best_params = optimizer.max['params']
    prophet_data = pd.DataFrame({'ds': train['date'], 'y': y_train})
    model_prophet = Prophet(
        yearly_seasonality=True,
        daily_seasonality=False,
        weekly_seasonality=True,
        seasonality_prior_scale=best_params['seasonality_prior_scale'],
        holidays_prior_scale=best_params['holidays_prior_scale'],
        changepoint_prior_scale=best_params['changepoint_prior_scale']
    )
    model_prophet.fit(prophet_data)
    future = model_prophet.make_future_dataframe(periods=len(test))
    forecast_prophet = model_prophet.predict(future)
    prophet_forecast = forecast_prophet['yhat'][-len(test):].values

        # Ensembling
    ensemble_forecast = (np.array(arima_forecast) + rfr_forecast + gbm_forecast + prophet_forecast) / 4
    mape_ensemble = custom_mape(y_test, ensemble_forecast)  # Note the change to 'custom_mape'
    accuracy_ensemble = 100 - mape_ensemble

    ensemble_results[sku] = {
        'MAPE': mape_ensemble,
        'Accuracy': accuracy_ensemble
    }

print(ensemble_results)





def interpret_dynamic_results(results):
    interpretations = {}

    # Categorize SKUs
    normal_skus = [sku for sku, metrics in results.items() if 0 <= metrics['Accuracy'] <= 1]
    abnormal_skus = [sku for sku, metrics in results.items() if not 0 <= metrics['Accuracy'] <= 1]

    for sku, metrics in results.items():
        mape = metrics['MAPE']
        accuracy = metrics['Accuracy']
        interpretation = ""

        if sku in abnormal_skus:
            interpretation += (f"For {sku}, the MAPE value is alarmingly high, suggesting the forecast might be producing extreme values. "
                               f"The 'Accuracy' is outside the standard range of 0% to 100%, indicating significant forecast errors. "
                               "The large discrepancy in the model's forecast suggests potential issues like extreme values, outliers, or "
                               "a model not suited for this data. Immediate reevaluation is recommended for this SKU.")
        elif sku in normal_skus:
            interpretation += (f"For {sku}, the MAPE is {mape:.4f}, indicating the predictions are, on average, off by about {mape*100:.2f}%. "
                               f"The forecast accuracy is {accuracy*100:.2f}%, which may or may not be satisfactory depending on the business context. "
                               "Given the positive accuracy value, this model is closer to being business-ready compared to others, but there's still room for improvement.")

        # How far into the future are they accurate?
        if sku in abnormal_skus:
            interpretation += ("\nBased on the forecast's significant discrepancy, it's safe to say that the predictions for this SKU are "
                               "unreliable even for the immediate future.")
        else:
            interpretation += ("\nGiven the positive accuracy value, the forecast seems more reliable for the immediate future, but "
                               "its reliability may decrease for extended horizons.")

        interpretations[sku] = interpretation

    return interpretations

detailed_interpretations = interpret_dynamic_results(ensemble_results)
for sku, text in detailed_interpretations.items():
    print(f"{sku}: {text}\n")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing SKU: SKU1
|   iter    |  target   |     d     |     p     |     q     |
-------------------------------------------------------------
| 1         | -1.201e+0 | 1.277     | 4.073     | 3.677     |
| 2         | -9.242e+0 | 2.604     | 1.917     | 4.897     |
| 3         | -8.425e+0 | 2.68      | 1.049     | 3.709     |
| 4         | -1.146e+0 | 1.989     | 4.434     | 4.29      |
| 5         | -4.463e+0 | 2.248     | 4.351     | 0.9338    |
| 6         | -1.172e+0 | 0.0       | 0.0       | 5.0       |
| 7         | -1.24e+09 | 0.0       | 5.0       | 5.0       |
| 8         | -1.159e+0 | 0.0       | 0.0       | 1.532     |
| 9         | -1.174e+0 | 0.0       | 2.488     | 5.0       |
| 10        | -2.28e+12 | 3.0       | 0.0       | 0.0       |
| 11        | -1.172e+0 | 0.0       | 5.0       | 0.0       |
| 12        | -1.163e+0 | 0.0       | 2.457 

DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/o_394pjr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/nj5ye5l4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12482', 'data', 'file=/tmp/tmpm4l3f5k1/o_394pjr.json', 'init=/tmp/tmpm4l3f5k1/nj5ye5l4.json', 'output', 'file=/tmp/tmpm4l3f5k1/prophet_modelqe0acm74/prophet_model-20231012004037.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:40:37 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:40:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


|   iter    |  target   | change... | holida... | season... |
-------------------------------------------------------------
| 1         | -3.748e+0 | 0.2134    | 8.148     | 7.357     |


DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/wugqtzpe.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/ncoozyhx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=78482', 'data', 'file=/tmp/tmpm4l3f5k1/wugqtzpe.json', 'init=/tmp/tmpm4l3f5k1/ncoozyhx.json', 'output', 'file=/tmp/tmpm4l3f5k1/prophet_model9yyua3rn/prophet_model-20231012004037.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:40:37 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:40:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/mehktes6.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/brpus1pj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

| 2         | -3.439e+0 | 0.4341    | 3.84      | 9.795     |
| 3         | -3.434e+0 | 0.4467    | 2.105     | 7.421     |


...

| 103       | -5.464e+0 | 0.4481    | 6.846     | 2.239     |


DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/umhkvyb7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/39c4_m6k.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=4843', 'data', 'file=/tmp/tmpm4l3f5k1/umhkvyb7.json', 'init=/tmp/tmpm4l3f5k1/39c4_m6k.json', 'output', 'file=/tmp/tmpm4l3f5k1/prophet_modelo473x1oa/prophet_model-20231012010605.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:06:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:06:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


| 104       | -5.309e+0 | 0.1368    | 5.58      | 8.721     |


DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/tmayluoi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/z06zu4gl.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34714', 'data', 'file=/tmp/tmpm4l3f5k1/tmayluoi.json', 'init=/tmp/tmpm4l3f5k1/z06zu4gl.json', 'output', 'file=/tmp/tmpm4l3f5k1/prophet_modelawx0ki_5/prophet_model-20231012010606.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:06:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:06:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/4d98f5o6.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4l3f5k1/p_aws1t5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

| 105       | -5.372e+0 | 0.2568    | 7.229     | 4.412     |
{'SKU1': {'MAPE': 1647681.2717655227, 'Accuracy': -1647581.2717655227}, 'SKU2': {'MAPE': 299203.142829213, 'Accuracy': -299103.142829213}, 'SKU3': {'MAPE': 1610205.7865024037, 'Accuracy': -1610105.7865024037}, 'SKU4': {'MAPE': 507888.4638358564, 'Accuracy': -507788.4638358564}, 'SKU5': {'MAPE': 353632.54354865255, 'Accuracy': -353532.54354865255}, 'SKU6': {'MAPE': 59.62359792253421, 'Accuracy': 40.37640207746579}}
SKU1: For SKU1, the MAPE value is alarmingly high, suggesting the forecast might be producing extreme values. The 'Accuracy' is outside the standard range of 0% to 100%, indicating significant forecast errors. The large discrepancy in the model's forecast suggests potential issues like extreme values, outliers, or a model not suited for this data. Immediate reevaluation is recommended for this SKU.
Based on the forecast's significant discrepancy, it's safe to say that the predictions for this SKU are unreliable eve